<a href="https://colab.research.google.com/github/luananova/Transforming-Comics-into-Accessible-Audio-Narratives/blob/main/NOTEBOOKS/Spell_Bubble_Generating_Accessibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using the OpenAI Text-to-Speech API
The goal is to transcribe texts into audio with narration and character dialogues

In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import os
import requests
import getpass
import openai
import json

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [ ]:
from google.colab import files

# Request the upload of the Associations file, from the "Extracting Texts" Notebook; and the Final_Page_Panel_Descriptions file, from the "Generating Descriptions" Notebook
uploaded_files = files.upload()

Saving associacoes.json to associacoes.json
Saving descricao_paineis_pagina_final.json to descricao_paineis_pagina_final.json


In [ ]:
# Load the JSON files
with open('descricao_paineis_pagina_final.json', 'r') as file:
    descricao_paineis = json.load(file)

with open('associacoes.json', 'r') as file:
    associacoes = json.load(file)

# Dictionary to store processed panels
painel_data_processado = {}

titulos_padronizados = {
    "Panel Type and Narrator's Caption": "Panel Type",
    "Panel Types": "Panel Type",
    "Panel Type & Narration": "Panel Type",
    "Scene Description": "Scene Setting",
    "Setting & Scene": "Scene Setting",
    "Characters and Actions": "Characters Interaction",
    "Characters and Dialogue": "Characters Interaction",
    "Characters & Dialogue": "Characters Interaction",
    "Character Interaction and Dialogue": "Characters Interaction",
    "Panel Description": "Description",
    "Interaction and Continuity": "Movement and Atmosphere",
    "Narrative Continuation": "Movement and Atmosphere"
}

numeros_para_palavras = {
    "painel1": "Panel One",
    "painel2": "Panel Two",
    "painel3": "Panel Three",
    "painel4": "Panel Four",
    "painel5": "Panel Five",
    "painel6": "Panel Six",
    "painel7": "Panel Seven"
}

# Function to normalize quotes in a string
def normalizar_aspas(texto):
    return texto.replace("\"", "'")

def padronizar_titulos(descricao):
    for antigo, novo in titulos_padronizados.items():
        descricao = descricao.replace(antigo, novo)

    # Remove hashtags
    descricao = descricao.replace("### ", "")
    return descricao

def adicionar_dois_pontos(titulo, descricao):
    if titulo in descricao and f"{titulo}:" not in descricao:
        descricao = descricao.replace(titulo, f"{titulo}:")
    return descricao

# Process each panel
for painel_key, painel_desc in descricao_paineis.items():

    # Convert the panel name (panel_key)
    painel_nome = numeros_para_palavras.get(painel_key, painel_key)

    personagens = associacoes[painel_key]["personagens"]

    # Replace character speeches in the narration
    for personagem, dados in personagens.items():
        fala = dados.get("fala")
        if fala:
            if painel_nome == "Panel One" and personagem == "Whiz":
                fala_normalizada = normalizar_aspas(fala)
                painel_desc_normalizado = normalizar_aspas(painel_desc)
                painel_desc = painel_desc_normalizado.replace(fala_normalizada, f"[Voz de {personagem}]: {fala}")
            else:
                painel_desc = painel_desc.replace(fala, f"[Voz de {personagem}]: {fala}")

    # Standardize titles
    painel_desc_padronizada = padronizar_titulos(painel_desc)

    # Add colons to specific titles
    for titulo in ["Panel Type", "Scene Setting", "Characters Interaction", "Movement and Atmosphere"]:
        painel_desc_padronizada = adicionar_dois_pontos(titulo, painel_desc_padronizada)

    painel_data_processado[painel_nome] = painel_desc_padronizada

    # Print the result for verification
    print(f"{painel_nome}:")
    print(painel_desc_padronizada)
    print("\n")  # Add a blank line between panels for better readability

# Here, 'processed_panel_data' contains the panels with replaced and standardized speeches

Panel One:
Panel Type:
The comic panel is a horizontal rectangle that prominently displays the caption 'Reunion in London.'

Scene Setting:
The scene is a mid-view of four characters, with a subdued background that suggests an indoor or urban environment, without specific details. The focus is on the character interactions rather than the surroundings.

Characters Interaction:
Daredevil, positioned on the left side of the panel, is depicted from the knees up and is engaged in a handshake with Silver Streak, who stands opposite him. Daredevil wears a red costume with a yellow belt. In the center of the panel, just behind Daredevil, is Churchill, presented from the chest up and dressed formally in a suit. He is a silent observer of the interaction. Silver Streak, visible from the knees up on the right, sports a yellow and red costume with an emblematic letter 'S' on his chest. Perched on Silver Streak's right shoulder is Whiz, a bird character, who adds a touch of blue to the panel's col

# Setting up different voices for the narration and each character

In [ ]:
# Save the mix of narration and voice in a JSON file
with open('painel_data_processado.json', 'w') as file:
    json.dump(painel_data_processado, file)

print(f"Mistura de narração e fala salva com sucesso:\n{painel_data_processado}\n")

Mistura de narração e fala salva com sucesso:
{'Panel One': 'Panel Type:\nThe comic panel is a horizontal rectangle that prominently displays the caption \'Reunion in London.\'\n\nScene Setting:\nThe scene is a mid-view of four characters, with a subdued background that suggests an indoor or urban environment, without specific details. The focus is on the character interactions rather than the surroundings.\n\nCharacters Interaction:\nDaredevil, positioned on the left side of the panel, is depicted from the knees up and is engaged in a handshake with Silver Streak, who stands opposite him. Daredevil wears a red costume with a yellow belt. In the center of the panel, just behind Daredevil, is Churchill, presented from the chest up and dressed formally in a suit. He is a silent observer of the interaction. Silver Streak, visible from the knees up on the right, sports a yellow and red costume with an emblematic letter \'S\' on his chest. Perched on Silver Streak\'s right shoulder is Whiz,

In [ ]:
import re

# Base voice settings for each character
vozes_base = {
    "Daredevil": "alloy",
    "Silver Streak": "onyx",
    "Whiz": "echo",
    "Churchill": "onyx",
    "O Policial": "alloy"
}

# Configuration adjustments for the narrator
voz_narrador = "fable"  # More neutral voice
pitch_narrador, speaking_rate_narrador, volume_gain_db_narrador = 0, 1.0, 0

# Function to create the text-to-speech setting
def criar_configuracao_tts(texto, voz, pitch, speaking_rate, volume_gain_db, stream=False):
    configuracao = {
        "input": texto,
        "model": "tts-1",
        "voice": voz,
        "audio_config": {
            "audio_encoding": "MP3",
            "speaking_rate": speaking_rate,
            "pitch": pitch,
            "volume_gain_db": volume_gain_db
        }
    }
    if stream:
        configuracao["stream"] = True
    return configuracao

# Function to adjust the voice setting based on the character and context
def ajustar_config_voz(personagem, fala, painel_nome):
    # If it's the narrator, return the default setting
    if personagem == "narrador":
        return {
            "voice": voz_narrador,
            "pitch": pitch_narrador,
            "speaking_rate": speaking_rate_narrador,
            "volume_gain_db": volume_gain_db_narrador
        }

    # Base settings for other characters
    voz_base = vozes_base[personagem]
    pitch, speaking_rate, volume_gain_db = 0, 1.2, 0

    # Adding adjustments for the characters in each panel
    # Adjustments for Panel 1
    if painel_nome == "Panel One":
        if personagem == "Daredevil":
            pitch = 3
        elif personagem == "Silver Streak":
            pitch = 0
            volume_gain_db = 1.5
        elif personagem == "Whiz":
            pitch = 7.5
            volume_gain_db = 1.5

    # Adjustments for Panel 2
    if painel_nome == "Panel Two":
        if personagem == "Churchill":
            pitch = -4
            speaking_rate = 0.85
            volume_gain_db = 3
        elif personagem == "Whiz":
            pitch = 7.5
            speaking_rate = 1.45
            volume_gain_db = 1.5

    # Adjustments for Panel 3
    if painel_nome == "Panel Three":
        if personagem == "Silver Streak":
            pitch = 0
        elif personagem == "Whiz":
            pitch = 7.5
            speaking_rate = 1.45
            volume_gain_db = 1.5
        elif personagem == "Churchill":
            pitch = -4
            speaking_rate = 0.85
            volume_gain_db = 3

    # Adjustments for Panel 4
    if painel_nome == "Panel Four":
        if personagem == "Silver Streak":
            pitch = 0
            volume_gain_db = 2
        elif personagem == "Churchill":
            pitch = -4
            speaking_rate = 0.85
            volume_gain_db = 3

    # Adjustments for Panel 5
    if painel_nome == "Panel Five":
        if personagem == "Silver Streak":
            pitch = 0
            volume_gain_db = 1.5
        elif personagem == "Whiz":
            pitch = 7.5
            speaking_rate = 1.45
            volume_gain_db = 1
        elif personagem == "O Policial":
            pitch = 0
            speaking_rate = 1.3
            volume_gain_db = 1.5

    # Adjustments for Panel 6
    if painel_nome == "Panel Six":
        if personagem == "Silver Streak":
            pitch = 0
            speaking_rate = 1.3
            volume_gain_db = 2.5
        elif personagem == "Whiz":
            pitch = 7.5
            speaking_rate = 1.45
            volume_gain_db = 1.5

    # Adjustments for Panel 7
    if painel_nome == "Panel Seven":
        if personagem == "Silver Streak":
            pitch = 0
            volume_gain_db = 1.5
        elif personagem == "Whiz":
            pitch = 7.5
            speaking_rate = 1.45
            volume_gain_db = 2.5

    return {
        "voice": voz_base,
        "pitch": pitch,
        "speaking_rate": speaking_rate,
        "volume_gain_db": volume_gain_db
    }

headers = {
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
    "Content-Type": "application/json"
}

def remove_character_dialogues(text, panel_name):
    # Defines a regular expression pattern to match character dialogues
    if panel_name == "Panel One":
        # Pattern for single quotes in "Panel One"
        pattern = r'\[Voz de [^\]]+\]: (?:(?!\[).)*\''
    elif panel_name in ["Panel Five", "Panel Six"]:
        # This pattern searches for "[Voice of ...]:" preceded by backslashes
        pattern = r'\[Voz de [^\]]+\]: [^\"\\]*(\\\"[^\"\\]*)*(?=\"|\[Voz de)'
    else:
        pattern = r'\[Voz de [^\]]+\]: (?:(?!\[).)*"'

    # Removes the characters' speeches from the text
    text = re.sub(pattern, '', text)

    # Removes isolated quotes that might have been left behind
    text = re.sub(r'(?<=\s)\'(?=\s|$)|(?<=\s)"(?=\s|$)', '', text)

    return text

# Processing the narration for each panel
for painel_nome, painel_desc_padronizada in painel_data_processado.items():
    descricao_sem_falas = remove_character_dialogues(painel_desc_padronizada, painel_nome)

    # Concatenate the panel name with the description
    texto_narracao = f"{painel_nome}. {descricao_sem_falas}"

    print(f"Descrição do {painel_nome} após remoção das falas dos personagens:")
    print(texto_narracao)
    print("\n")

    # Count characters and check if streaming is needed
    num_caracteres = len(texto_narracao)
    precisa_stream = num_caracteres > 4096
    print(f"Número de caracteres na narração do {painel_nome}: {num_caracteres}")

    configuracao_tts_narrador = criar_configuracao_tts(texto_narracao, voz_narrador, pitch_narrador, speaking_rate_narrador, volume_gain_db_narrador, stream=precisa_stream)
    response = requests.post("https://api.openai.com/v1/audio/speech", headers=headers, data=json.dumps(configuracao_tts_narrador))

    # Check if the response is successful
    if response.status_code == 200:
        nome_arquivo = f"{painel_nome}_narracao.mp3"
        with open(nome_arquivo, 'wb') as audio_file:
            audio_file.write(response.content)
    else:
        print(f"Erro na geração de áudio para a narração no {painel_nome}: {response.status_code}")
        print("Detalhes do erro:", response.text)

# Processing the characters' speeches
for painel_key, painel in associacoes.items():
    for personagem, dados in painel["personagens"].items():
        fala = dados.get("fala")
        if fala:
            # Count characters and check if streaming is needed
            num_caracteres = len(fala)
            precisa_stream = num_caracteres > 4096
            print(f"Número de caracteres na fala de {personagem}: {num_caracteres}")
            print(f"Character: {personagem}, Panel: {painel_key}, Text: {fala}")

            configuracao_voz_personagem = ajustar_config_voz(personagem, fala, painel_key)
            configuracao_tts_personagem = criar_configuracao_tts(fala, configuracao_voz_personagem["voice"], configuracao_voz_personagem["pitch"], configuracao_voz_personagem["speaking_rate"], configuracao_voz_personagem["volume_gain_db"], stream=precisa_stream)
            # Here you call the API to generate the audio of the character's speech

            response = requests.post("https://api.openai.com/v1/audio/speech", headers=headers, data=json.dumps(configuracao_tts_personagem))

            # Check if the response is successful
            if response.status_code == 200:
                nome_arquivo = f"{painel_key}_{personagem}.mp3"
                with open(nome_arquivo, 'wb') as audio_file:
                    audio_file.write(response.content)
            else:
                print(f"Erro na geração de áudio para {personagem} no {painel_key}: {response.status_code}")
                print("Detalhes do erro:", response.text)

Descrição do Panel One após remoção das falas dos personagens:
Panel One. Panel Type:
The comic panel is a horizontal rectangle that prominently displays the caption 'Reunion in London.'

Scene Setting:
The scene is a mid-view of four characters, with a subdued background that suggests an indoor or urban environment, without specific details. The focus is on the character interactions rather than the surroundings.

Characters Interaction:
Daredevil, positioned on the left side of the panel, is depicted from the knees up and is engaged in a handshake with Silver Streak, who stands opposite him. Daredevil wears a red costume with a yellow belt. In the center of the panel, just behind Daredevil, is Churchill, presented from the chest up and dressed formally in a suit. He is a silent observer of the interaction. Silver Streak, visible from the knees up on the right, sports a yellow and red costume with an emblematic letter 'S' on his chest. Perched on Silver Streak's right shoulder is Whiz

# Concatenating the audio generated per panel
The goal is to insert the dialogues into the narrations using timestamps to use more than one type of voice and have a single file in the end

In [ ]:
pip install pydub

In [ ]:
pip install whisper-timestamped

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 851.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.5/645.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 20.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=21b1994d80fb723bfdd5c80b305ecbf6bae7a77f93d3217b94821876d92dd172
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed

In [ ]:
from pydub import AudioSegment
import whisper_timestamped as whisper

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [ ]:
# Defining a function to transcribe the audios and get the timestamps
def transcribe_audio_and_get_word_timestamps(audio_path):
    model = whisper.load_model("base")
    audio = whisper.load_audio(audio_path)
    result = whisper.transcribe(model, audio)

    # Extracting the transcribed texts and the timestamps for each word
    words = []
    for segment in result["segments"]:
        for word in segment.get("words", []):
            words.append((word["start"], word["end"], word["text"]))

    return result["text"], words

# Step 1: Transcription of the narration audio - PANEL 1
narration_audio_path = "/content/Panel One_narracao.mp3"
narration_text, narration_word_timestamps = transcribe_audio_and_get_word_timestamps(narration_audio_path)

# Loading the audio segments
narration_audio = AudioSegment.from_mp3("Panel One_narracao.mp3")
daredevil_audio = AudioSegment.from_mp3("painel1_Daredevil.mp3")
silver_streak_audio = AudioSegment.from_mp3("painel1_Silver Streak.mp3")
whiz_audio = AudioSegment.from_mp3("painel1_Whiz.mp3")

Detected language: English


100%|██████████| 9196/9196 [00:04<00:00, 2017.18frames/s]


In [ ]:
# Printing the results to see the timestamps
for start, end, text in narration_word_timestamps:
    print(f"Start: {start}s, End: {end}s, Word: {text}")

Start: 0.04s, End: 0.24s, Word: Panel
Start: 0.24s, End: 0.56s, Word: 1.
Start: 1.0s, End: 1.64s, Word: Panel
Start: 1.64s, End: 2.0s, Word: type.
Start: 2.76s, End: 2.98s, Word: The
Start: 2.98s, End: 3.24s, Word: comic
Start: 3.24s, End: 3.58s, Word: panel
Start: 3.58s, End: 3.8s, Word: is
Start: 3.8s, End: 3.92s, Word: a
Start: 3.92s, End: 4.34s, Word: horizontal
Start: 4.34s, End: 4.96s, Word: rectangle
Start: 4.96s, End: 5.32s, Word: that
Start: 5.32s, End: 5.84s, Word: prominently
Start: 5.84s, End: 6.26s, Word: displays
Start: 6.36s, End: 6.56s, Word: the
Start: 6.56s, End: 6.88s, Word: caption
Start: 6.88s, End: 7.32s, Word: reunion
Start: 7.32s, End: 7.58s, Word: in
Start: 7.58s, End: 7.88s, Word: London.
Start: 8.7s, End: 9.12s, Word: Seen
Start: 9.12s, End: 9.5s, Word: setting,
Start: 9.92s, End: 10.02s, Word: the
Start: 10.02s, End: 10.22s, Word: scene
Start: 10.22s, End: 10.38s, Word: is
Start: 10.38s, End: 10.5s, Word: a
Start: 10.5s, End: 10.88s, Word: mid-view
Start: 10

In [ ]:
def generate_silence(duration_milliseconds):
    return AudioSegment.silent(duration=duration_milliseconds)

In [ ]:
# Insertion points based on guesses (in milliseconds)
# These are the points where you are certain or have an approximate estimate
dialogue_insert_points_ms = [
    int(65.72 * 1000),  # End of the word "with" and beginning of the word "Silver"
]

# Timestamps for calculating midpoint
# These are the timestamps for the end of one word and the beginning of the next
calculated_timestamps = [
    (66.96, 67.84), # Between "responds" and "Whiz"
    (70.42, 71.22) # Between "remark" and "Graphic"
    # Add more pairs of timestamps for calculation here
]

# Calculates the midpoint between the end of one word and the beginning of the next
for (end_previous, start_next) in calculated_timestamps:
    insertion_point = ((end_previous + start_next) / 2) * 1000  # Converting to milliseconds
    dialogue_insert_points_ms.append(int(insertion_point))

# Sorting the final list to ensure chronological sequence
dialogue_insert_points_ms.sort()

dialogue_insert_points_ms

# Function to insert dialogues into the narration with pauses before and after each dialogue
def insert_dialogues_with_pauses(narration_audio, dialogue_audios, dialogue_insert_points_ms, pause_duration):
    final_audio = narration_audio
    current_offset_ms = 0

    for i, dialogue_audio in enumerate(dialogue_audios):
        # Calculating the new insertion point considering the current offset
        insert_point_ms = dialogue_insert_points_ms[i] + current_offset_ms

        # Splitting the narration at the insertion point
        before_dialogue = final_audio[:insert_point_ms]
        after_dialogue = final_audio[insert_point_ms:]

        # Concatenating the narration with a pause, dialogue, and another pause
        final_audio = before_dialogue + generate_silence(pause_duration) + dialogue_audio + generate_silence(pause_duration) + after_dialogue

        # Updating the offset for the next insertion
        # Considering the duration of the dialogue and the two pauses
        current_offset_ms += len(dialogue_audio) + 2 * pause_duration

    return final_audio

# List of dialogue audios
dialogue_audios = [daredevil_audio, silver_streak_audio, whiz_audio]

# Duration of the pause (e.g., one second)
pause_duration = 1000  # In milliseconds

# Insert the dialogues into the narration with pauses before and after each dialogue
final_audio = insert_dialogues_with_pauses(narration_audio, dialogue_audios, dialogue_insert_points_ms, pause_duration)

# Save the final audio
final_audio.export("panel_1_final.mp3", format="mp3")

<_io.BufferedRandom name='panel_one_final.mp3'>

# Repeating the process for the other panels of the page

In [ ]:
# Mapping panel numbers to full names lol
panel_names = {
    2: "Two",
    3: "Three",
    4: "Four",
    5: "Five",
    6: "Six",
    7: "Seven"
}

# Function to transcribe the narration
def transcribe_narration(audio_path):
    # Load the Whisper model
    model = whisper.load_model("base")

    narration_audio = whisper.load_audio(audio_path)
    narration_result = whisper.transcribe(model, narration_audio)

    # Extract timestamps for each word
    word_timestamps = [(word["start"], word["end"], word["text"])
                       for segment in narration_result["segments"]
                       for word in segment.get("words", [])]

    return narration_result["text"], word_timestamps

# Dictionary to store the texts and timestamps of each panel
narration_texts = {}
narration_timestamps = {}
narration_audios = {}

# Loop to process each panel
for panel_number in range(2, 8):  # From 2 to 7
    panel_name = panel_names[panel_number]

    # Specify the audio file path
    narration_audio_path = f"/content/Panel {panel_name}_narracao.mp3"

    # Load the audio segment
    narration_audio = AudioSegment.from_mp3(narration_audio_path)
    narration_audios[panel_number] = narration_audio

     # Transcribe the narration
    text, timestamps = transcribe_narration(narration_audio_path)
    narration_texts[panel_number] = text
    narration_timestamps[panel_number] = timestamps

# Now you have the texts, timestamps, and narration audios for each panel stored

Detected language: English


100%|██████████| 7968/7968 [00:03<00:00, 2073.56frames/s]


Detected language: English


100%|██████████| 5359/5359 [00:02<00:00, 1932.04frames/s]


Detected language: English


...

100%|██████████| 5582/5582 [00:02<00:00, 2087.43frames/s]


Detected language: English


100%|██████████| 4898/4898 [00:02<00:00, 1874.72frames/s]


Detected language: English


100%|██████████| 4488/4488 [00:02<00:00, 1829.46frames/s]


In [ ]:
# Access and print the narration timestamps of the panels, according to the index adjustment
for start, end, text in narration_timestamps[7]:
    print(f"Start: {start}s, End: {end}s, Word: {text}")

Start: 0.08s, End: 0.26s, Word: Panel
Start: 0.26s, End: 0.64s, Word: 7.
Start: 1.06s, End: 1.44s, Word: Panel
Start: 1.44s, End: 1.76s, Word: type,
Start: 2.02s, End: 2.46s, Word: vertical,
Start: 2.92s, End: 3.42s, Word: narrators
Start: 3.42s, End: 3.86s, Word: caption.
Start: 4.66s, End: 4.8s, Word: A
Start: 4.8s, End: 5.12s, Word: carrier
Start: 5.12s, End: 5.44s, Word: pigeon
Start: 5.44s, End: 5.82s, Word: flies
Start: 5.82s, End: 6.1s, Word: from
Start: 6.1s, End: 6.24s, Word: the
Start: 6.24s, End: 6.56s, Word: lighted
Start: 6.56s, End: 6.88s, Word: window.
Start: 7.26s, End: 7.42s, Word: Wiz
Start: 7.42s, End: 7.66s, Word: takes
Start: 7.66s, End: 7.94s, Word: off
Start: 7.94s, End: 8.18s, Word: from
Start: 8.18s, End: 8.48s, Word: Silver
Start: 8.48s, End: 8.9s, Word: Streaks
Start: 8.9s, End: 9.3s, Word: shoulder.
Start: 10.24s, End: 10.5s, Word: Scene
Start: 10.5s, End: 10.92s, Word: setting.
Start: 11.48s, End: 11.62s, Word: The
Start: 11.62s, End: 11.88s, Word: action
S

In [ ]:
# Information for each panel based on the timestamps
panel_info = {
    2: {
        "calculated_timestamps": [(21.86, 22.3), (56.8, 57.84)], # Between "stating" and "his" // Between "assertion" and "Whiz's"
        "character_names": ["Churchill", "Whiz"]
    },
    3: {
        "calculated_timestamps": [(24.88, 25.24), (34.62, 35.98), (47.56, 48.18)],  # Between "says" and "his" // Between "with" and "this" // Between "question" and "his"
        "character_names": ["Silver Streak", "Whiz", "Churchill"]
    },
    4: {
        "calculated_timestamps": [(18.9, 19.34), (28.3, 28.78)],  # Between "declares" and "his" // Between "tone" and "his"
        "character_names": ["Silver Streak", "Churchill"]
    },
    5: {
        "calculated_timestamps": [(20.68, 21.08), (26.28, 27.06), (34.2, 34.38)], # Between "companion" and "Whiz" // Between "confidently" and "below" // Between "exclaiming" and "scene"
        "character_names": ["Silver Streak", "Whiz", "O Policial"]
    },
    6: {
        "calculated_timestamps": [(32.62, 33.56), (37.88, 38.74)],  # Between "says" and "acknowledging" // Between "determination" and "movement"
        "character_names": ["Silver Streak", "Whiz"]
    },
    7: {
        "palpite_timestamps": [22000, 24540],  # Between "commands" and "responding" // Between "declares" and "movement"
        "character_names": ["Silver Streak", "Whiz"]
    }
}

character_audios = {}

# Loop to calculate the insertion points per panel
for panel_number in range(2, 8):  # De 2 a 7
    info = panel_info.get(panel_number, {})
    # Calculate insertion points
    dialogue_insert_points_ms = info.get("palpite_timestamps", [])
    for (end_previous, start_next) in info.get("calculated_timestamps", []):
        insertion_point = ((end_previous + start_next) / 2) * 1000
        dialogue_insert_points_ms.append(int(insertion_point))
    dialogue_insert_points_ms.sort()

    # Specify the path of the narration audio file
    narration_audio_path = f"/content/Panel {panel_names[panel_number]}_narracao.mp3"

    # Load the specific narration audio segment file for this panel
    narration_audio = AudioSegment.from_mp3(narration_audio_path)

    # Load the character dialogue files
    dialogue_audios = [AudioSegment.from_mp3(f"/content/painel{panel_number}_{name}.mp3") for name in info["character_names"]]
    character_audios[panel_number] = dialogue_audios

    # Insert the dialogues into the narration
    final_audio = insert_dialogues_with_pauses(narration_audio, dialogue_audios, dialogue_insert_points_ms, pause_duration)

    # Save the final audio
    output_filename = f"panel_{panel_number}_final.mp3"
    final_audio.export(output_filename, format="mp3")

    print(f"Áudio final para o painel {panel_number} foi salvo como {output_filename}")

Áudio final para o painel 2 foi salvo como panel_2_final.mp3
Áudio final para o painel 3 foi salvo como panel_3_final.mp3
Áudio final para o painel 4 foi salvo como panel_4_final.mp3
Áudio final para o painel 5 foi salvo como panel_5_final.mp3
Áudio final para o painel 6 foi salvo como panel_6_final.mp3
Áudio final para o painel 7 foi salvo como panel_7_final.mp3
